In [1]:
library(rvest)
library(dplyr)
library(stringr)
library(rvest)
library(RSelenium)
library(countrycode)



Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'stringr' was built under R version 3.6.3"Warning message:
"package 'RSelenium' was built under R version 3.6.3"

# RSelenium
To run selenium first, I needed to download a web driver for chrome and a selenium program. Then using the command line, I run a line of code to start the web driver and selenium program. "java -Dwebdriver.chrome.driver="C:\Users/hayden\Downloads\chromedriver.exe" -jar selenium-server-standalone-3.141.59.jar". I used the port number the selenium server was running on to connect it to R using the R selenium package.  

In [2]:
# This code creates a connection to the selenium server
con <- remoteDriver(remoteServerAddr = "localhost", 
                    port = 4444,
                    browserName = "chrome"
)

In [3]:
#this code opens up the connection
con$open()

[1] "Connecting to remote server"
$acceptInsecureCerts
[1] FALSE

$browserName
[1] "chrome"

$browserVersion
[1] "94.0.4606.81"

$chrome
$chrome$chromedriverVersion
[1] "94.0.4606.61 (418b78f5838ed0b1c69bb4e51ea0252171854915-refs/branch-heads/4606@{#1204})"

$chrome$userDataDir
[1] "C:\\Users\\hayde\\AppData\\Local\\Temp\\scoped_dir4932_66342293"


$`goog:chromeOptions`
$`goog:chromeOptions`$debuggerAddress
[1] "localhost:53340"


$networkConnectionEnabled
[1] FALSE

$pageLoadStrategy
[1] "normal"

$platformName
[1] "windows"

$proxy
named list()

$setWindowRect
[1] TRUE

$strictFileInteractability
[1] FALSE

$timeouts
$timeouts$implicit
[1] 0

$timeouts$pageLoad
[1] 300000

$timeouts$script
[1] 30000


$unhandledPromptBehavior
[1] "dismiss and notify"

$`webauthn:extension:credBlob`
[1] TRUE

$`webauthn:extension:largeBlob`
[1] TRUE

$`webauthn:virtualAuthenticators`
[1] TRUE

$webdriver.remote.sessionid
[1] "27e9e90f7d1e6a96b99a145335ed97ac"

$id
[1] "27e9e90f7d1e6a96b99a145335ed97ac

In [4]:
#this code navigates the Selenium server to the desired webpage
con$navigate("https://www.cia.gov/the-world-factbook/field/urbanization/")

In [5]:
#this code extracts the HTML code from the server
urban_html <- con$getPageSource()[[1]]

In [6]:
#this code reads in the contents of the HTML file to be used by rvest
page <- read_html(urban_html)

In [7]:
#This code extracts the country name from the HTML script
country <- page %>% html_nodes(".content-area-content a") %>% html_text()
country

[1] "Afghanistan"                                  
  [2] "Albania"                                      
  [3] "Algeria"                                      
  [4] "American Samoa"                               
  [5] "Andorra"                                      
  [6] "Angola"                                       
  [7] "Anguilla"                                     
  [8] "Antigua and Barbuda"                          
  [9] "Argentina"                                    
 [10] "Armenia"                                      
 [11] "Aruba"                                        
 [12] "Australia"                                    
 [13] "Austria"                                      
 [14] "Azerbaijan"                                   
 [15] "Bahamas, The"                                 
 [16] "Bahrain"                                      
 [17] "Bangladesh"                                   
 [18] "Barbados"                                     
 [19] "Belarus"                                      
 [20] "Belgium"                                      
 [21] "Belize"                                       
 [22] "Benin"                                        
 [23] "Bermuda"                                      
 [24] "Bhutan"                                       
 [25] "Bolivia"                                      
 [26] "Bosnia and Herzegovina"                       
 [27] "Botswana"                                     
 [28] "Brazil"                                       
 [29] "British Virgin Islands"                       
 [30] "Brunei"                                       
 [31] "Bulgaria"                                     
 [32] "Burkina Faso"                                 
 [33] "Burma"                                        
 [34] "Burundi"                                      
 [35] "Cabo Verde"                                   
 [36] "Cambodia"                                     
 [37] "Cameroon"                                     
 [38] "Canada"                                       
 [39] "Cayman Islands"                               
 [40] "Central African Republic"                     
 [41] "Chad"                                         
 [42] "Chile"                                        
 [43] "China"                                        
 [44] "Colombia"                                     
 [45] "Comoros"                                      
 [46] "Congo, Democratic Republic of the"            
 [47] "Congo, Republic of the"                       
 [48] "Cook Islands"                                 
 [49] "Costa Rica"                                   
 [50] "Cote d'Ivoire"                                
 [51] "Croatia"                                      
 [52] "Cuba"                                         
 [53] "Curacao"                                      
 [54] "Cyprus"                                       
 [55] "Czechia"                                      
 [56] "Denmark"                                      
 [57] "Djibouti"                                     
 [58] "Dominica"                                     
 [59] "Dominican Republic"                           
 [60] "Ecuador"                                      
 [61] "Egypt"                                        
 [62] "El Salvador"                                  
 [63] "Equatorial Guinea"                            
 [64] "Eritrea"                                      
 [65] "Estonia"                                      
 [66] "Eswatini"                                     
 [67] "Ethiopia"                                     
 [68] "Falkland Islands (Islas Malvinas)"            
 [69] "Faroe Islands"                                
 [70] "Fiji"                                         
 [71] "Finland"                                      
 [72] "France"                                       
 [73] "French Polynesia"                             
 [74] "Gabon"                                        
 [75] 

In [8]:
#This code extracts the sentence that contains the urbanisation rate and the yearly change in urbanisation
country_urban_rate <- page %>% html_nodes("li p") %>% html_text()

In [9]:
#This code extracts the variables urbanisation rate and the yearly change in urbanisation from the sentence
urbanisation_rate <-  str_match(country_urban_rate, "urban population:\\s*(.*?)\\s*% of total population")[,2]
urbanisation_yearly_change <-  str_match(country_urban_rate, "rate of urbanization:\\s*(.*?)\\s*% annual rate of change")[,2]

In [10]:
#This code combines the 3 variables into a data frame
urban_df <- data.frame(country, urbanisation_rate, urbanisation_yearly_change)
urban_df

country                  urbanisation_rate urbanisation_yearly_change
1   Afghanistan              26.3              3.34                      
2   Albania                  63                1.29                      
3   Algeria                  74.3              1.99                      
4   American Samoa           87.2              0.26                      
5   Andorra                  87.9              0.11                      
6   Angola                   67.5              4.04                      
7   Anguilla                 100               0.47                      
8   Antigua and Barbuda      24.4              0.87                      
9   Argentina                92.2              0.97                      
10  Armenia                  63.4              0.23                      
11  Aruba                    43.9              0.77                      
12  Australia                86.4              1.27                      
13  Austria                  59                0.68                      
14  Azerbaijan               56.8              1.38                      
15  Bahamas, The             83.4              1.02                      
16  Bahrain                  89.6              1.99                      
17  Bangladesh               38.9              2.88                      
18  Barbados                 31.2              0.46                      
19  Belarus                  79.9              0.28                      
20  Belgium                  98.1              0.38                      
21  Belize                   46.2              2.3                       
22  Benin                    49                3.74                      
23  Bermuda                  100               -0.2                      
24  Bhutan                   43                2.52                      
25  Bolivia                  70.5              1.87                      
26  Bosnia and Herzegovina   49.4              0.61                      
27  Botswana                 71.6              2.47                      
28  Brazil                   87.3              0.87                      
29  British Virgin Islands   48.9              1.73                      
30  Brunei                   78.6              1.44                      
... ...                      ...               ...                       
201 Tajikistan               27.7              2.73                      
202 Tanzania                 36                4.89                      
203 Thailand                 52.2              1.43                      
204 Timor-Leste              31.7              3.31                      
205 Togo                     43.4              3.6                       
206 Tokelau                  0                 0                         
207 Tonga                    23.1              0.99                      
208 Trinidad and Tobago      53.3              0.23                      
209 Tunisia                  69.9              1.34                      
210 Turkey                   76.6              1.11                      
211 Turkmenistan             53                2.23                      
212 Turks and Caicos Islands 93.8              1.46                      
213 Tuvalu                   64.8              2.08                      
214 Uganda                   25.6              5.41                      
215 Ukraine                  69.8              -0.27                     
216 United Arab Emirates     87.3              1.5                       
217 United Kingdom           84.2              0.8                       
218 United States            82.9              0.96                      
219 Uruguay                  95.6              0.4                       
220 Uzbekistan               50.4              1.25                      
221 Vanuatu                  25.7              2.55                      
222 Venezuela                88.3              1.16                      
223 Viet

In [11]:
#This code gets the HTML script for the population variable
con$navigate("https://www.cia.gov/the-world-factbook/field/population/country-comparison?fbclid=IwAR2jG4pZYN8gcvnSppY9VOROoqQIiAA48EIVy24LKpwlEpoHHTNJJcalVmU")
population_html <- con$getPageSource()[[1]]


In [12]:
#This code extracts the table containing the population variable from the HTML script
pop_page <- read_html(population_html)
pop_table <- pop_page %>% html_node("table") %>% html_table()
colnames(pop_table) <- c("rank", "country", "population", "date_of_Information")
pop_table

rank country                                       population   
1    1   China                                         1,397,897,720
2    2   India                                         1,339,330,514
3    3   United States                                 334,998,398  
4    4   Indonesia                                     275,122,131  
5    5   Pakistan                                      238,181,034  
6    6   Nigeria                                       219,463,862  
7    7   Brazil                                        213,445,417  
8    8   Bangladesh                                    164,098,818  
9    9   Russia                                        142,320,790  
10  10   Mexico                                        130,207,371  
11  11   Japan                                         124,687,293  
12  12   Ethiopia                                      110,871,031  
13  13   Philippines                                   110,818,325  
14  14   Egypt                                         106,437,241  
15  15   Congo, Democratic Republic of the             105,044,646  
16  16   Vietnam                                       102,789,598  
17  17   Iran                                          85,888,910   
18  18   Turkey                                        82,482,383   
19  19   Germany                                       79,903,481   
20  20   Thailand                                      69,480,520   
21  21   France                                        68,084,217   
22  22   United Kingdom                                67,081,000   
23  23   Italy                                         62,390,364   
24  24   Tanzania                                      62,092,761   
25  25   Burma                                         57,069,099   
26  26   South Africa                                  56,978,635   
27  27   Kenya                                         54,685,051   
28  28   Korea, South                                  51,715,162   
29  29   Colombia                                      50,355,650   
30  30   Spain                                         47,260,584   
... ...  ...                                           ...          
207 208  Faroe Islands                                 51,943       
208 209  Northern Mariana Islands                      51,659       
209 210  American Samoa                                46,366       
210 211  Sint Maarten                                  44,564       
211 212  Liechtenstein                                 39,425       
212 213  British Virgin Islands                        37,891       
213 214  San Marino                                    34,467       
214 215  Saint Martin                                  32,680       
215 216  Monaco                                        31,223       
216 217  Gibraltar                                     29,516       
217 218  Palau                                         21,613       
218 219  Anguilla                                      18,403       
219 220  Wallis and Futuna                             15,851       
220 221  Tuvalu                                        11,448       
221 222  Nauru                                         9,770        
222 223  Cook Islands                                  8,327        
223 224  Saint Helena, Ascension, and Tristan da Cunha 7,915        
224 225  Saint Barthelemy                              7,116        
225 226  Montserrat                                    5,387        
226 227  Saint Pierre and Miquelon                     5,321        
227 228  Falkland Islands (Islas Malvinas)             3,198        
228 229  Svalbard                                      2,926        
229 230  Christmas Island                              2,205        
230 231  Niue                                          2,000        
231 232  Norfolk Island                                1,748        
232 233  Tokelau                                       1,647        
23

In [13]:
#This code makes the population variable a numeric value by removing commas
pop_table$population <- as.numeric(gsub(",","",pop_table$population))
pop_table

rank country                                       population
1    1   China                                         1397897720
2    2   India                                         1339330514
3    3   United States                                  334998398
4    4   Indonesia                                      275122131
5    5   Pakistan                                       238181034
6    6   Nigeria                                        219463862
7    7   Brazil                                         213445417
8    8   Bangladesh                                     164098818
9    9   Russia                                         142320790
10  10   Mexico                                         130207371
11  11   Japan                                          124687293
12  12   Ethiopia                                       110871031
13  13   Philippines                                    110818325
14  14   Egypt                                          106437241
15  15   Congo, Democratic Republic of the              105044646
16  16   Vietnam                                        102789598
17  17   Iran                                            85888910
18  18   Turkey                                          82482383
19  19   Germany                                         79903481
20  20   Thailand                                        69480520
21  21   France                                          68084217
22  22   United Kingdom                                  67081000
23  23   Italy                                           62390364
24  24   Tanzania                                        62092761
25  25   Burma                                           57069099
26  26   South Africa                                    56978635
27  27   Kenya                                           54685051
28  28   Korea, South                                    51715162
29  29   Colombia                                        50355650
30  30   Spain                                           47260584
... ...  ...                                           ...       
207 208  Faroe Islands                                 51943     
208 209  Northern Mariana Islands                      51659     
209 210  American Samoa                                46366     
210 211  Sint Maarten                                  44564     
211 212  Liechtenstein                                 39425     
212 213  British Virgin Islands                        37891     
213 214  San Marino                                    34467     
214 215  Saint Martin                                  32680     
215 216  Monaco                                        31223     
216 217  Gibraltar                                     29516     
217 218  Palau                                         21613     
218 219  Anguilla                                      18403     
219 220  Wallis and Futuna                             15851     
220 221  Tuvalu                                        11448     
221 222  Nauru                                          9770     
222 223  Cook Islands                                   8327     
223 224  Saint Helena, Ascension, and Tristan da Cunha  7915     
224 225  Saint Barthelemy                               7116     
225 226  Montserrat                                     5387     
226 227  Saint Pierre and Miquelon                      5321     
227 228  Falkland Islands (Islas Malvinas)              3198     
228 229  Svalbard                                       2926     
229 230  Christmas Island                               2205     
230 231  Niue                                           2000     
231 232  Norfolk Island                                 1748     
232 233  Tokelau                                        1647     
233 234  Paracel Islands                                1440     
234 235  Holy See (Vatican City)                        1000     
235 236  Cocos (Keeling) Islands            

In [14]:
# This code gets HTML script containing the size variable

con$navigate("https://www.cia.gov/the-world-factbook/field/area/country-comparison")
size_html <- con$getPageSource()[[1]]

In [15]:
#This code extracts the table containing the size variable from the HTML script

size_page <- read_html(size_html)
size_table <- size_page %>% html_node("table") %>% html_table() %>% select(1:3)
colnames(size_table) <- c("rank", "country", "size")
size_table

rank country                           size      
1    1   Russia                            17,098,242
2    2   Antarctica                        14,200,000
3    3   Canada                            9,984,670 
4    4   United States                     9,833,517 
5    5   China                             9,596,960 
6    6   Brazil                            8,515,770 
7    7   Australia                         7,741,220 
8    8   India                             3,287,263 
9    9   Argentina                         2,780,400 
10  10   Kazakhstan                        2,724,900 
11  11   Algeria                           2,381,740 
12  12   Congo, Democratic Republic of the 2,344,858 
13  13   Greenland                         2,166,086 
14  14   Saudi Arabia                      2,149,690 
15  15   Mexico                            1,964,375 
16  16   Indonesia                         1,904,569 
17  17   Sudan                             1,861,484 
18  18   Libya                             1,759,540 
19  19   Iran                              1,648,195 
20  20   Mongolia                          1,564,116 
21  21   Peru                              1,285,216 
22  22   Chad                              1,284,000 
23  23   Niger                             1,267,000 
24  24   Angola                            1,246,700 
25  25   Mali                              1,240,192 
26  26   South Africa                      1,219,090 
27  27   Colombia                          1,138,910 
28  28   Ethiopia                          1,104,300 
29  29   Bolivia                           1,098,581 
30  30   Mauritania                        1,030,700 
... ...  ...                               ...       
227 228  San Marino                        61        
228 229  British Indian Ocean Territory    60        
229 230  Saint Martin                      54        
230 231  Bermuda                           54        
231 232  Bouvet Island                     49        
232 233  Pitcairn Islands                  47        
233 234  Norfolk Island                    36        
234 235  Sint Maarten                      34        
235 236  Macau                             28        
236 237  Tuvalu                            26        
237 238  Saint Barthelemy                  25        
238 239  Nauru                             21        
239 240  Cocos (Keeling) Islands           14        
240 241  Tokelau                           12        
241 242  Palmyra Atoll                     12        
242 243  Paracel Islands                   8         
243 244  Gibraltar                         7         
244 245  Wake Island                       7         
245 246  Midway Islands                    6         
246 247  Clipperton Island                 6         
247 248  Navassa Island                    5         
248 249  Ashmore and Cartier Islands       5         
249 250  Spratly Islands                   5         
250 251  Jarvis Island                     5         
251 252  Coral Sea Islands                 3         
252 253  Johnston Atoll                    3         
253 254  Monaco                            2         
254 255  Howland Island                    2         
255 256  Kingman Reef                      1         
256 257  Holy See (Vatican City)           0

In [16]:
#This code makes the size variable a numeric value by removing the commas
size_table$size <- as.numeric(gsub(",","",size_table$size))
size_table

rank country                           size    
1    1   Russia                            17098242
2    2   Antarctica                        14200000
3    3   Canada                             9984670
4    4   United States                      9833517
5    5   China                              9596960
6    6   Brazil                             8515770
7    7   Australia                          7741220
8    8   India                              3287263
9    9   Argentina                          2780400
10  10   Kazakhstan                         2724900
11  11   Algeria                            2381740
12  12   Congo, Democratic Republic of the  2344858
13  13   Greenland                          2166086
14  14   Saudi Arabia                       2149690
15  15   Mexico                             1964375
16  16   Indonesia                          1904569
17  17   Sudan                              1861484
18  18   Libya                              1759540
19  19   Iran                               1648195
20  20   Mongolia                           1564116
21  21   Peru                               1285216
22  22   Chad                               1284000
23  23   Niger                              1267000
24  24   Angola                             1246700
25  25   Mali                               1240192
26  26   South Africa                       1219090
27  27   Colombia                           1138910
28  28   Ethiopia                           1104300
29  29   Bolivia                            1098581
30  30   Mauritania                         1030700
... ...  ...                               ...     
227 228  San Marino                        61      
228 229  British Indian Ocean Territory    60      
229 230  Saint Martin                      54      
230 231  Bermuda                           54      
231 232  Bouvet Island                     49      
232 233  Pitcairn Islands                  47      
233 234  Norfolk Island                    36      
234 235  Sint Maarten                      34      
235 236  Macau                             28      
236 237  Tuvalu                            26      
237 238  Saint Barthelemy                  25      
238 239  Nauru                             21      
239 240  Cocos (Keeling) Islands           14      
240 241  Tokelau                           12      
241 242  Palmyra Atoll                     12      
242 243  Paracel Islands                    8      
243 244  Gibraltar                          7      
244 245  Wake Island                        7      
245 246  Midway Islands                     6      
246 247  Clipperton Island                  6      
247 248  Navassa Island                     5      
248 249  Ashmore and Cartier Islands        5      
249 250  Spratly Islands                    5      
250 251  Jarvis Island                      5      
251 252  Coral Sea Islands                  3      
252 253  Johnston Atoll                     3      
253 254  Monaco                             2      
254 255  Howland Island                     2      
255 256  Kingman Reef                       1      
256 257  Holy See (Vatican City)            0

In [17]:
#This code gets the HTML script for the GDP per capita variable
con$navigate("https://www.cia.gov/the-world-factbook/field/real-gdp-per-capita/country-comparison")
gdp_html <- con$getPageSource()[[1]]

In [18]:
#This code extracts the table containing the real GDP per capita variable from the HTML script
gdp_page = read_html(gdp_html)
GDP_table <- gdp_page %>% html_node("table") %>% html_table()
names(GDP_table) <- c("rank", "country", "gdp_per_cap", "date")
GDP_table

rank country                           gdp_per_cap date     
1    1   Liechtenstein                     $139,100    2009 est.
2    2   Macau                             $123,965    2019 est.
3    3   Monaco                            $115,700    2015 est.
4    4   Luxembourg                        $114,482    2019 est.
5    5   Singapore                         $97,341     2019 est.
6    6   Qatar                             $90,044     2019 est.
7    7   Ireland                           $86,781     2019 est.
8    8   Isle of Man                       $84,600     2014 est.
9    9   Bermuda                           $81,798     2019 est.
10  10   Cayman Islands                    $71,549     2018 est.
11  11   Falkland Islands (Islas Malvinas) $70,800     2015 est.
12  12   Switzerland                       $68,628     2019 est.
13  13   United Arab Emirates              $67,119     2019 est.
14  14   Norway                            $63,633     2019 est.
15  15   United States                     $62,530     2019 est.
16  16   Brunei                            $62,100     2019 est.
17  17   Gibraltar                         $61,700     2014 est.
18  18   Hong Kong                         $59,848     2019 est.
19  19   San Marino                        $59,439     2018 est.
20  20   Denmark                           $57,804     2019 est.
21  21   Netherlands                       $56,935     2019 est.
22  22   Jersey                            $56,600     2016 est.
23  23   Austria                           $56,188     2019 est.
24  24   Iceland                           $55,874     2019 est.
25  25   Germany                           $53,919     2019 est.
26  26   Sweden                            $53,240     2019 est.
27  27   Guernsey                          $52,500     2014 est.
28  28   Belgium                           $51,934     2019 est.
29  29   Andorra                           $49,900     2015 est.
30  30   Australia                         $49,854     2019 est.
... ...  ...                               ...         ...      
198 199  Syria                             $2,900      2015 est.
199 200  Zimbabwe                          $2,836      2019 est.
200 201  Lesotho                           $2,704      2019 est.
201 202  Solomon Islands                   $2,663      2019 est.
202 203  Tanzania                          $2,660      2019 est.
203 204  Guinea                            $2,562      2019 est.
204 205  Yemen                             $2,500      2017 est.
205 206  Mali                              $2,322      2019 est.
206 207  Kiribati                          $2,272      2019 est.
207 208  Rwanda                            $2,227      2019 est.
208 209  Gambia, The                       $2,223      2019 est.
209 210  Ethiopia                          $2,221      2019 est.
210 211  Uganda                            $2,187      2019 est.
211 212  Burkina Faso                      $2,178      2019 est.
212 213  Afghanistan                       $2,065      2019 est.
213 214  Guinea-Bissau                     $1,989      2019 est.
214 215  Sierra Leone                      $1,718      2019 est.
215 216  Korea, North                      $1,700      2015 est.
216 217  Madagascar                        $1,647      2019 est.
217 218  Eritrea                           $1,600      2017 est.
218 219  South Sudan                       $1,600      2017 est.
219 220  Togo                              $1,597      2019 est.
220 221  Chad                              $1,580      2019 est.
221 222  Liberia                           $1,428      2019 est.
222 223  Mozambique                        $1,281      2019 est.
223 224  Niger                             $1,225      2019 est.
224 225  Congo, Democratic Republic of the $1,098      2019 est.
225 226  Malawi                            $1,060      2019 est.
226 227  Central African Republic          $945        2019 est.
227 228  Burundi                       

In [19]:
#This code makes the size variable a numeric value by removing the commas and the dollar sign
GDP_table$gdp_per_cap <- GDP_table$gdp_per_cap %>%
    gsub(",","", .) %>%
    gsub("\\$", "", .) %>%
    as.integer()
GDP_table


rank country                           gdp_per_cap date     
1    1   Liechtenstein                     139100      2009 est.
2    2   Macau                             123965      2019 est.
3    3   Monaco                            115700      2015 est.
4    4   Luxembourg                        114482      2019 est.
5    5   Singapore                          97341      2019 est.
6    6   Qatar                              90044      2019 est.
7    7   Ireland                            86781      2019 est.
8    8   Isle of Man                        84600      2014 est.
9    9   Bermuda                            81798      2019 est.
10  10   Cayman Islands                     71549      2018 est.
11  11   Falkland Islands (Islas Malvinas)  70800      2015 est.
12  12   Switzerland                        68628      2019 est.
13  13   United Arab Emirates               67119      2019 est.
14  14   Norway                             63633      2019 est.
15  15   United States                      62530      2019 est.
16  16   Brunei                             62100      2019 est.
17  17   Gibraltar                          61700      2014 est.
18  18   Hong Kong                          59848      2019 est.
19  19   San Marino                         59439      2018 est.
20  20   Denmark                            57804      2019 est.
21  21   Netherlands                        56935      2019 est.
22  22   Jersey                             56600      2016 est.
23  23   Austria                            56188      2019 est.
24  24   Iceland                            55874      2019 est.
25  25   Germany                            53919      2019 est.
26  26   Sweden                             53240      2019 est.
27  27   Guernsey                           52500      2014 est.
28  28   Belgium                            51934      2019 est.
29  29   Andorra                            49900      2015 est.
30  30   Australia                          49854      2019 est.
... ...  ...                               ...         ...      
198 199  Syria                             2900        2015 est.
199 200  Zimbabwe                          2836        2019 est.
200 201  Lesotho                           2704        2019 est.
201 202  Solomon Islands                   2663        2019 est.
202 203  Tanzania                          2660        2019 est.
203 204  Guinea                            2562        2019 est.
204 205  Yemen                             2500        2017 est.
205 206  Mali                              2322        2019 est.
206 207  Kiribati                          2272        2019 est.
207 208  Rwanda                            2227        2019 est.
208 209  Gambia, The                       2223        2019 est.
209 210  Ethiopia                          2221        2019 est.
210 211  Uganda                            2187        2019 est.
211 212  Burkina Faso                      2178        2019 est.
212 213  Afghanistan                       2065        2019 est.
213 214  Guinea-Bissau                     1989        2019 est.
214 215  Sierra Leone                      1718        2019 est.
215 216  Korea, North                      1700        2015 est.
216 217  Madagascar                        1647        2019 est.
217 218  Eritrea                           1600        2017 est.
218 219  South Sudan                       1600        2017 est.
219 220  Togo                              1597        2019 est.
220 221  Chad                              1580        2019 est.
221 222  Liberia                           1428        2019 est.
222 223  Mozambique                        1281        2019 est.
223 224  Niger                             1225        2019 est.
224 225  Congo, Democratic Republic of the 1098        2019 est.
225 226  Malawi                            1060        2019 est.
226 227  Central African Republic           945        2019 est.
227 228  Burundi                       

In [20]:
# this code joins together the urban_df, GDP_table, size_table, pop_table. This also code removes all the rows 
#that do not have value for all variables as all the united nation recognised states are 
#included in the final data set while the remainders are Proto-states. 

final_df <- merge(urban_df, GDP_table[,c(2,3)], by= "country")
final_df <- merge(final_df, size_table[, c(2,3)], by= "country")
final_df <- merge(final_df, pop_table[, c(2,3)], by= "country")
str(final_df)

'data.frame':	225 obs. of  6 variables:
 $ country                   : Factor w/ 230 levels "Afghanistan",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ urbanisation_rate         : Factor w/ 187 levels "0","100","13.5",..: 20 95 126 160 164 106 2 15 175 97 ...
 $ urbanisation_yearly_change: Factor w/ 177 levels "-0.03","-0.11",..: 145 80 110 24 16 160 37 59 66 22 ...
 $ gdp_per_cap               : int  2065 13965 11511 11200 49900 6670 12200 21910 22064 13654 ...
 $ size                      : num  652230 28748 2381740 224 468 ...
 $ population                : num  37466414 3088385 43576691 46366 85645 ...


In [24]:
#This code creates a vector of country codes using countrycode function. 
country_code <- countrycode(final_df$country, 'country.name', 'iso3c')

In [26]:
country_code

[1] "AFG" "ALB" "DZA" "ASM" "AND" "AGO" "AIA" "ATG" "ARG" "ARM" "ABW" "AUS"
 [13] "AUT" "AZE" "BHS" "BHR" "BGD" "BRB" "BLR" "BEL" "BLZ" "BEN" "BMU" "BTN"
 [25] "BOL" "BIH" "BWA" "BRA" "VGB" "BRN" "BGR" "BFA" "MMR" "BDI" "CPV" "KHM"
 [37] "CMR" "CAN" "CYM" "CAF" "TCD" "CHL" "CHN" "COL" "COM" "COD" "COG" "COK"
 [49] "CRI" "CIV" "HRV" "CUB" "CUW" "CYP" "CZE" "DNK" "DJI" "DMA" "DOM" "ECU"
 [61] "EGY" "SLV" "GNQ" "ERI" "EST" "SWZ" "ETH" "FLK" "FRO" "FJI" "FIN" "FRA"
 [73] "PYF" "GAB" "GMB" "PSE" "GEO" "DEU" "GHA" "GIB" "GRC" "GRL" "GRD" "GUM"
 [85] "GTM" "GGY" "GIN" "GNB" "GUY" "HTI" "HND" "HKG" "HUN" "ISL" "IND" "IDN"
 [97] "IRN" "IRQ" "IRL" "IMN" "ISR" "ITA" "JAM" "JPN" "JEY" "JOR" "KAZ" "KEN"
[109] "KIR" "PRK" "KOR" "KWT" "KGZ" "LAO" "LVA" "LBN" "LSO" "LBR" "LBY" "LIE"
[121] "LTU" "LUX" "MAC" "MDG" "MWI" "MYS" "MDV" "MLI" "MLT" "MHL" "MRT" "MUS"
[133] "MEX" "FSM" "MDA" "MCO" "MNG" "MNE" "MSR" "MAR" "MOZ" "NAM" "NRU" "NPL"
[145] "NLD" "NCL" "NZL" "NIC" "NER" "NGA" "NIU" "MNP" "NOR" "OMN" "PAK" "PLW"
[157] "PAN" "PNG" "PRY" "PER" "PHL" "POL" "PRT" "PRI" "QAT" "ROU" "RUS" "RWA"
[169] "SHN" "KNA" "LCA" "SPM" "VCT" "WSM" "SMR" "STP" "SAU" "SEN" "SRB" "SYC"
[181] "SLE" "SGP" "SXM" "SVK" "SVN" "SLB" "ZAF" "SSD" "ESP" "LKA" "SDN" "SUR"
[193] "SWE" "CHE" "SYR" "TWN" "TJK" "TZA" "THA" "TLS" "TGO" "TKL" "TON" "TTO"
[205] "TUN" "TUR" "TKM" "TCA" "TUV" "UGA" "UKR" "ARE" "GBR" "USA" "URY" "UZB"
[217] "VUT" "VEN" "VNM" NA    "WLF" "PSE" "YEM" "ZMB" "ZWE"

In [27]:
#this script creates the density variable by dividing size by population
final_df <- final_df %>%
    mutate(density = round(population / size ,2) )
final_df

country                  urbanisation_rate urbanisation_yearly_change
1   Afghanistan              26.3              3.34                      
2   Albania                  63                1.29                      
3   Algeria                  74.3              1.99                      
4   American Samoa           87.2              0.26                      
5   Andorra                  87.9              0.11                      
6   Angola                   67.5              4.04                      
7   Anguilla                 100               0.47                      
8   Antigua and Barbuda      24.4              0.87                      
9   Argentina                92.2              0.97                      
10  Armenia                  63.4              0.23                      
11  Aruba                    43.9              0.77                      
12  Australia                86.4              1.27                      
13  Austria                  59                0.68                      
14  Azerbaijan               56.8              1.38                      
15  Bahamas, The             83.4              1.02                      
16  Bahrain                  89.6              1.99                      
17  Bangladesh               38.9              2.88                      
18  Barbados                 31.2              0.46                      
19  Belarus                  79.9              0.28                      
20  Belgium                  98.1              0.38                      
21  Belize                   46.2              2.3                       
22  Benin                    49                3.74                      
23  Bermuda                  100               -0.2                      
24  Bhutan                   43                2.52                      
25  Bolivia                  70.5              1.87                      
26  Bosnia and Herzegovina   49.4              0.61                      
27  Botswana                 71.6              2.47                      
28  Brazil                   87.3              0.87                      
29  British Virgin Islands   48.9              1.73                      
30  Brunei                   78.6              1.44                      
... ...                      ...               ...                       
196 Taiwan                   79.3              0.65                      
197 Tajikistan               27.7              2.73                      
198 Tanzania                 36                4.89                      
199 Thailand                 52.2              1.43                      
200 Timor-Leste              31.7              3.31                      
201 Togo                     43.4              3.6                       
202 Tokelau                  0                 0                         
203 Tonga                    23.1              0.99                      
204 Trinidad and Tobago      53.3              0.23                      
205 Tunisia                  69.9              1.34                      
206 Turkey                   76.6              1.11                      
207 Turkmenistan             53                2.23                      
208 Turks and Caicos Islands 93.8              1.46                      
209 Tuvalu                   64.8              2.08                      
210 Uganda                   25.6              5.41                      
211 Ukraine                  69.8              -0.27                     
212 United Arab Emirates     87.3              1.5                       
213 United Kingdom           84.2              0.8                       
214 United States            82.9              0.96                      
215 Uruguay                  95.6              0.4                       
216 Uzbekistan               50.4              1.25                      
217 Vanuatu                  25.7              2.55                      
218 Vene

In [ ]:
#This code combines the country code into final data frame
final_df <- cbind(final_df,country_code)

In [31]:
final_df 

X   country                  urbanisation_rate urbanisation_yearly_change
1    1  Afghanistan               26.3              3.34                     
2    2  Albania                   63.0              1.29                     
3    3  Algeria                   74.3              1.99                     
4    4  American Samoa            87.2              0.26                     
5    5  Andorra                   87.9              0.11                     
6    6  Angola                    67.5              4.04                     
7    7  Anguilla                 100.0              0.47                     
8    8  Antigua and Barbuda       24.4              0.87                     
9    9  Argentina                 92.2              0.97                     
10  10  Armenia                   63.4              0.23                     
11  11  Aruba                     43.9              0.77                     
12  12  Australia                 86.4              1.27                     
13  13  Austria                   59.0              0.68                     
14  14  Azerbaijan                56.8              1.38                     
15  15  Bahamas, The              83.4              1.02                     
16  16  Bahrain                   89.6              1.99                     
17  17  Bangladesh                38.9              2.88                     
18  18  Barbados                  31.2              0.46                     
19  19  Belarus                   79.9              0.28                     
20  20  Belgium                   98.1              0.38                     
21  21  Belize                    46.2              2.30                     
22  22  Benin                     49.0              3.74                     
23  23  Bermuda                  100.0             -0.20                     
24  24  Bhutan                    43.0              2.52                     
25  25  Bolivia                   70.5              1.87                     
26  26  Bosnia and Herzegovina    49.4              0.61                     
27  27  Botswana                  71.6              2.47                     
28  28  Brazil                    87.3              0.87                     
29  29  British Virgin Islands    48.9              1.73                     
30  30  Brunei                    78.6              1.44                     
... ... ...                      ...               ...                       
196 196 Taiwan                   79.3               0.65                     
197 197 Tajikistan               27.7               2.73                     
198 198 Tanzania                 36.0               4.89                     
199 199 Thailand                 52.2               1.43                     
200 200 Timor-Leste              31.7               3.31                     
201 201 Togo                     43.4               3.60                     
202 202 Tokelau                   0.0               0.00                     
203 203 Tonga                    23.1               0.99                     
204 204 Trinidad and Tobago      53.3               0.23                     
205 205 Tunisia                  69.9               1.34                     
206 206 Turkey                   76.6               1.11                     
207 207 Turkmenistan             53.0               2.23                     
208 208 Turks and Caicos Islands 93.8               1.46                     
209 209 Tuvalu                   64.8               2.08                     
210 210 Uganda                   25.6               5.41                     
211 211 Ukraine                  69.8              -0.27                     
212 212 United Arab Emirates     87.3               1.50                     
213 213 United Kingdom           84.2               0.80                     
214 214 United States            82.9               0.96                     
215 215 Uruguay           

In [32]:
#This code creates a CSV file
write.csv(final_df, "DataFrame.csv")

In [29]:
#This code closes the connection to the selenium server
con$close()